In [1]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# --- Sample Dummy Data ---
# Replace this with real NFL standings data
teams = ['49ers', 'Cowboys', 'Eagles', 'Chiefs', 'Bills']
weeks = list(range(1, 19))

# Create dummy win-loss data
data = []
for week in weeks:
    for team in teams:
        wins = min(week, max(0, int(week / 2 + (hash(team + str(week)) % 3 - 1))))
        losses = week - wins
        data.append({'Week': week, 'Team': team, 'Wins': wins, 'Losses': losses})

df = pd.DataFrame(data)

# --- Plotting Function ---
def plot_standings(week):
    week_data = df[df['Week'] == week]
    fig = px.bar(
        week_data,
        x='Team',
        y='Wins',
        color='Team',
        title=f'NFL Team Wins - Week {week}',
        labels={'Wins': 'Number of Wins'},
        height=500
    )
    fig.update_layout(showlegend=False)
    fig.show()

# --- Slider Widget ---
week_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=18,
    step=1,
    description='Week:',
    continuous_update=False
)

# --- Link Slider to Plot ---
widgets.interact(plot_standings, week=week_slider)

interactive(children=(IntSlider(value=1, continuous_update=False, description='Week:', max=18, min=1), Output(…

<function __main__.plot_standings(week)>

In [8]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
from dash import dash_table

# Dummy data generator for testing
def generate_dummy_data():
    import numpy as np
    teams = {
        "AFC East": ["Bills", "Dolphins", "Patriots", "Jets"],
        "AFC North": ["Ravens", "Bengals", "Browns", "Steelers"],
        "AFC South": ["Texans", "Colts", "Jaguars", "Titans"],
        "AFC West": ["Chiefs", "Broncos", "Raiders", "Chargers"],
        "NFC East": ["Eagles", "Cowboys", "Giants", "Commanders"],
        "NFC North": ["Lions", "Vikings", "Packers", "Bears"],
        "NFC South": ["Buccaneers", "Saints", "Falcons", "Panthers"],
        "NFC West": ["49ers", "Rams", "Seahawks", "Cardinals"]
    }

    data = []
    for week in range(1, 19):  # Weeks 1-18
        for division, team_list in teams.items():
            for team in team_list:
                wins = np.random.randint(0, week+1)
                losses = week - wins
                pf = np.random.randint(150, 350)
                pa = np.random.randint(150, 350)
                data.append({
                    "Week": week,
                    "Team": team,
                    "Division": division,
                    "Wins": wins,
                    "Losses": losses,
                    "PointsFor": pf,
                    "PointsAgainst": pa
                })

    return pd.DataFrame(data)

# Load or generate your weekly NFL data
df = generate_dummy_data()

# Initialize the Dash app
app = dash.Dash(__name__)
app.title = "NFL 2024 Standings by Week"

# Layout
app.layout = html.Div([
    html.H1("NFL 2024 Division Standings"),
    
    dcc.Slider(
        id='week-slider',
        min=1,
        max=18,
        step=1,
        value=1,
        marks={i: f"Week {i}" for i in range(1, 19)},
    ),
    
    html.Div(id='standings-output')
], style={'width': '80%', 'margin': 'auto'})


# Callback to update standings
@app.callback(
    Output('standings-output', 'children'),
    Input('week-slider', 'value')
)
def update_standings(week):
    filtered = df[df['Week'] == week].copy()

    # Calculate win percentage
    filtered['WinPct'] = filtered['Wins'] / (filtered['Wins'] + filtered['Losses']).replace(0, 1)

    # Sort by WinPct, PointsFor (desc), PointsAgainst (asc)
    filtered.sort_values(
        by=['Division', 'WinPct', 'PointsFor', 'PointsAgainst'],
        ascending=[True, False, False, True],
        inplace=True
    )

    output = []
    for division in filtered['Division'].unique():
        div_df = filtered[filtered['Division'] == division]
        table = dash_table.DataTable(
            columns=[
                {"name": "Team", "id": "Team"},
                {"name": "W", "id": "Wins"},
                {"name": "L", "id": "Losses"},
                {"name": "Win %", "id": "WinPct", "type": "numeric", "format": {'specifier': '.3f'}},
                {"name": "PF", "id": "PointsFor"},
                {"name": "PA", "id": "PointsAgainst"},
            ],
            data=div_df.to_dict('records'),
            style_table={'overflowX': 'auto'},
            style_header={'fontWeight': 'bold'},
            style_cell={'textAlign': 'center'},
        )
        output.append(html.H3(division))
        output.append(table)

    return output

# Run app
if __name__ == "__main__":
    app.run(debug=True)

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run.

In [9]:
pip install nfl_data_py

Note: you may need to restart the kernel to use updated packages.


In [10]:
from nfl_data_py import import_pbp_data
import pandas as pd

# Get play-by-play data for the full 2024 season (once it's available)
pbp = import_pbp_data([2024])

# Save a copy to avoid repeated downloads
pbp.to_csv("nfl_2024_pbp.csv", index=False)

2024 done.
Downcasting floats.


In [11]:
weekly_summary = (
    pbp[pbp['play_type'].notna()]  # remove non-plays
    .groupby(['week', 'posteam'])
    .agg({
        'posteam_score': 'max',
        'defteam_score': 'max',
    })
    .reset_index()
)